In [1]:
import torch
import torch.nn as nn
import mmap
import random
import pickle
from torch.nn import functional as F
import os.path

device = 'cuda' if torch.cuda.is_available() else 'cpu'

batch_size = 64
block_size = 128
max_iters = 3000
# learning_rate = 3e-3, 3e-4, 1e-3, 1e-4
learning_rate = 3e-4
eval_iters = 100
dropout = 0.2 # dropout random neurons in the network to avoid overfitting
n_embd = 384
# embedding_vector = [0.1, 0.2, 0.8, 1.1] # ^^would be 384 elements long in the vector
n_head = 16
n_layer = 16

How many decoder layers do we have?<br>
Could have Sequential neural network with 4 decoder layers (or decoder blocks)

In [2]:
chars = ""
with open('openwebtext/vocab.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))

vocab_size = len(chars)

In [3]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])



In [4]:
# We will use memory mapping to get the text that we need
def get_random_chunk(split):
    filename = "openwebtext/train_split.txt" if split == 'train' else "openwebtext/val_split.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # determine file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random positiona and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')

            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)
    return data


def get_batch(split):
    data = get_random_chunk(split)
    # initialize ix - rand integer between 0 and lenght of text - block_size
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    # Stack them in batches
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train() # put the model into training mode
    return out

In [7]:
class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size(batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x) # (B,T,hs)
        q = self.query(x) #(B,T,hs)
        # compute attention scores(affinities)
        # transpose puts the matrices into the correct shape that they can be multiplied correctly
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) #(B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation o fthe values
        v = self.value(x) # (B, T, hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

# masked fill: [1, 0.6, 0.4]
# mask out everything except for the first time-step
# on the next time-step then expose the next piece
# [1, 0, 0] -> [1, 0.6, 0] -> [1, 0.6, 0.4]

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd) # an extra learnable parameters
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # Last dimension (B x T x F(Features)) -> (B, T, [h1, h1, h1, h2, h2, h2, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

class FeedForward(nn.Module):
    """ a simple linear layer followed by a non-linearity """
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """
    # Each of these decoder blocks is a block from the diagram model
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # n_embd -> we want the tokens to be in a really large vector
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, index, targets=None):
        B, T = index.shape
        tok_emb = self.token_embedding_table(index) # (B, T, C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T, C)
        # Broadcasting semantics - rules about how you perform arithmetic operations on a tensor
        x = tok_emb + pos_emb # (B, T, C) shape
        x = self.blocks(x) # (B, T, C)
        x = self.ln_f(x) # (B, T, C)
        logits = self.lm_head(x) # (B, T, vocab_size)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss

    def generate(self, index, max_new_tokens):
        # index is (B,T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            # there are not just one or two normalizations - there are many of them
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sampled index to the running sequence
            index = torch.cat((index,index_next), dim=1) #(B, T+1)
        return index

model = GPTLanguageModel(vocab_size)
m = model.to(device)

if os.path.isfile('model-01.pkl'):
    print('loading model parameters...')
    with open('model-01.pkl', 'rb') as f:
        model = pickle.load(f)
    print('loaded successfully')
    m = model.to(device)
else:
    pass

loading model parameters...
loaded successfully


# activation functions
nn.Linear() in a row
Would apply one transformation that would multiply themselves together
So what can we do that is more than linear transformations
3 activation functions will help with the computational part of this
functional.relu

In [8]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}") # single quotes mean braces and brackets are the same
    # sample a batch of data
    xb, yb = get_batch('train')
    # evaluate the loss
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

with open('model-01.pkl', 'wb') as f:
    pickle.dump(model, f)

step: 0, train loss: 1.291, val loss: 1.169
step: 100, train loss: 1.194, val loss: 1.240
step: 200, train loss: 1.236, val loss: 1.218
step: 300, train loss: 1.184, val loss: 1.221
step: 400, train loss: 1.242, val loss: 1.204
step: 500, train loss: 1.215, val loss: 1.216
step: 600, train loss: 1.210, val loss: 1.257
step: 700, train loss: 1.215, val loss: 1.206
step: 800, train loss: 1.241, val loss: 1.241
step: 900, train loss: 1.220, val loss: 1.206
step: 1000, train loss: 1.201, val loss: 1.253
step: 1100, train loss: 1.237, val loss: 1.174
step: 1200, train loss: 1.199, val loss: 1.181
step: 1300, train loss: 1.253, val loss: 1.244
step: 1400, train loss: 1.255, val loss: 1.203
step: 1500, train loss: 1.255, val loss: 1.290
step: 1600, train loss: 1.220, val loss: 1.215
step: 1700, train loss: 1.202, val loss: 1.212
step: 1800, train loss: 1.174, val loss: 1.191
step: 1900, train loss: 1.187, val loss: 1.207
step: 2000, train loss: 1.214, val loss: 1.195
step: 2100, train loss: 1

We are looking at the Standard Deviation function<br>
sum(xi) = 0.6 from arr = [0.1,0.2,0.3]<br>
std deviation of: [-0.38, 0.52, 2.48]<br>
N = 3 (number of elements)<br>
mean = average of sum / N<br>
mean = 0.87333333333<br>
sq_rt{1/3 * sigma(xi - 0.87)^2}<br>
std_dev = ~1.19<br>
When initializing parameters - if you have outliers in your data they will really stretch out the standard deviation and you will not learn anything<br>
If you have data too close together then they all learn the same thing, you need a balanced and stable network.<br>
@4:40 in the video look at how he is breaking apart the large file you downloaded<br>
OpenWeb Text which OpenAI used to train its model<br>
https://skylion007.github.io/OpenWebTextCorpus/<br>
<br>
As we scale up we will use torch.load and torch.save to store those parameters in the model-01 file that we create and load<br>


Final Notes:<br>
It doesn't have useful output right now.  However, with another course in fine tuning we could.  But this builds on the whole architecture and gives<br>
you a good idea of how you would build a simple chatbot.<br>

- efficiency testing<br>
- RNNs - recurrent neural networks<br>
    - sequential learning<br>
    - a little dumber than transformers<br>
    - runs on CPU<br>
- Quantization - reduce the memory usage by your parameters<br>
    - Paper: QLoRA: Efficient Finetuning of Quantized LLMs<br>